In [1]:
%cd /content/drive/My Drive/Colab Notebooks/AMEX/ResponseEngineDataUpdate_v2

import os

# import sys
# sys.path.append('01_utils')
# from getdata import *
# from hlpr import *
# from processdata import *

from datetime import datetime
import pandas as pd
import numpy as np
# import requests
# import time
# import json

/content/drive/My Drive/Colab Notebooks/AMEX/ResponseEngineDataUpdate_v2


In [2]:
# paths
UTILS_PATH = '01_utils'
DATA_PATH = '02_data'

# temp market map
mkt_map = {'AU':'Australia','GB':'United Kingdom','US':'United States','CA':'Canada','MX':'Mexico'}

In [3]:
# import category mapping
df_map = pd.read_excel(os.path.join(UTILS_PATH, 'map.xlsx'), sheet_name='terms')

In [4]:
df_map.iloc[:, :-1]

,Master Category,Category,term
0,Additional,Cultural Trends,baking
1,Additional,Cultural Trends,cooking
2,Additional,Cultural Trends,diy
3,Additional,Cultural Trends,exercise
4,Additional,Cultural Trends,games
...,...,...,...
332,Lifestyle,Dining Out,locally owned restaurants
333,Lifestyle,Dining Out,best local eats
334,Lifestyle,Dining Out,local dining
335,Lifestyle,Dining In / Delivery,takeout restaurants


In [5]:
# temp market map
mkt_map = {'AU':'Australia','GB':'United Kingdom','US':'United States','CA':'Canada','MX':'Mexico'}

# import google search results
GOOGLE_PATH = os.path.join(DATA_PATH, 'google')

df_list = []
for f in os.listdir(GOOGLE_PATH):
  for d in os.listdir(os.path.join(GOOGLE_PATH, f)):
    df = pd.read_csv(os.path.join(GOOGLE_PATH, f, d))
    df_list.append(df)

# combine
df = pd.concat(df_list, sort=False).reset_index(drop=True)

# adjust columns
df['date'] = pd.to_datetime(df['date'])
df.rename(columns={'variable':'term'}, inplace=True)

# join to category map
df_join = mastermap_['google_categories'].merge(df, how='left')

# create the aggregated view for the trend tab 
df_join['day_of_year'] = df_join['date'].dt.dayofyear

# group data
df_grp = df_join.groupby(['date', 'day_of_year', 'market', 'Master Category', 'Category'])['value'].mean().reset_index()

# add historical value
df_grp_hist = df_grp[df_grp['date'].dt.year < 2020].groupby(['day_of_year', 'market', 'master_category', 'category'])['value'].mean().reset_index()
df_grp_hist.rename(columns={'value':'value_3yr'}, inplace=True)

# filter to current >= 2020
df_grp_curr = df_grp[df_grp['date'].dt.year >= 2020].reset_index(drop=True)

# set preliminary data
df_prelim = df_grp_curr.merge(df_grp_hist, how='left', on=['day_of_year', 'market', 'master_category', 'category'])

# set market columns
df_prelim['market_name'] = df_prelim['market'].apply(lambda x: mkt_map[x])

# column order
cols = ['date', 'market','market_name','master_category','category','value','value_3yr']
df_fnl = df_prelim[cols]

NameError: ignored

,date,market,market_name,master_category,category,value,value_3yr
0,2020-01-01,AU,Australia,Additional,Cultural Trends,26.615385,35.404762
1,2020-01-01,AU,Australia,Amex,Brand,16.090909,18.083333
2,2020-01-01,AU,Australia,Coronavirus/Health,Coronavirus,1.571429,18.714286
3,2020-01-01,AU,Australia,Economy,Economy,1.500000,10.166667
4,2020-01-01,AU,Australia,Entertainment,Streaming,65.000000,88.666667
...,...,...,...,...,...,...,...
39670,2020-12-10,US,United States,Small Business,B2B,38.071429,30.309524
39671,2020-12-10,US,United States,Small Business,Demand,42.666667,33.703704
39672,2020-12-10,US,United States,Small Business,Financial,38.833333,31.833333
39673,2020-12-10,US,United States,Small Business,Running A Business,25.666667,25.444444
